In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
import pickle
import seaborn as sns
import os

import vdmlab as vdm

from load_data import get_pos, get_spikes, get_lfp
from tuning_curves_functions import get_tc_1d, get_odd_firing_idx

import info.R063d2_info as r063d2
import info.R063d3_info as r063d3
import info.R063d4_info as r063d4
import info.R063d5_info as r063d5
import info.R063d6_info as r063d6
import info.R066d1_info as r066d1
import info.R066d2_info as r066d2
import info.R066d3_info as r066d3
import info.R066d4_info as r066d4
import info.R067d1_info as r067d1
import info.R067d2_info as r067d2

In [ ]:
pickle_filepath = 'E:\\code\\python-vdmlab\\projects\\emily_shortcut\\cache\\pickled\\'
output_filepath = 'E:\\code\\python-vdmlab\\projects\\emily_shortcut\\plots\\'
# pickle_filepath = 'C:\\Users\\Emily\\Code\\python-vdmlab\\projects\\emily_shortcut\\cache\\pickled\\'
# output_filepath = 'C:\\Users\\Emily\\Code\\python-vdmlab\\projects\\emily_shortcut\\plots\\'

In [ ]:
info = r063d4

In [ ]:
print(info.session_id)
pos = get_pos(info.pos_mat, info.pxl_to_cm)
csc = get_lfp(info.good_swr[0])
spikes = get_spikes(info.spike_mat)

In [ ]:
y = np.array([[0.0, 1.0],
              [1.6, 2.0]])

w = np.array([[0.5, 1.2],
              [2.0, 2.2]])

In [ ]:
def intersect(this, other):
    new_starts = []
    new_stops = []
    for y in this:
        for w in other:
            if (y[0] <= w[0] and y[1] >= w[1]) or (y[0] <= w[0] and y[1] < w[1] and y[1] > w[0]) or \
                (y[0] > w[0] and y[1] >= w[1] and y[0] < w[1]) or (y[0] > w[0] and y[1] < w[1]):
                new_starts.append(y[0])
                new_stops.append(y[1])
    
    return np.hstack((np.array(new_starts)[..., np.newaxis],
                      np.array(new_stops)[..., np.newaxis]))

intersects = intersect(y, w)
intersects

In [ ]:
def union(yarray, warray):
    new_starts = []
    new_stops = []
    for y in yarray:
        for w in warray:
            if y[0] <= w[0] and y[1] >= w[1]:
                new_starts.append(w[0])
                new_stops.append(w[1])
            elif y[0] <= w[0] and y[1] < w[1] and y[1] > w[0]:
                new_starts.append(w[0])
                new_stops.append(y[1])
            elif y[0] > w[0] and y[1] >= w[1] and y[0] < w[1]:
                new_starts.append(y[0])
                new_stops.append(w[1])
            elif y[0] > w[0] and y[1] < w[1]:
                new_starts.append(y[0])
                new_stops.append(y[1])
                
    return np.hstack(
            (np.array(new_starts)[..., np.newaxis],
             np.array(new_stops)[..., np.newaxis]))
    
merged = union(y, w)
print(merged)

In [ ]:
import numpy as np

ep1 = np.array([[0.0, 1.0],
                [0.9, 1.5],
                [1.6, 2.0]])

def merge(ep, gap=0.0):
    ends = ep[:-1, 1] + gap
    starts = ep[1:, 0]
    to_merge = (ends - starts) >= 0
    
    new_starts = [ep[0, 0]]
    new_ends = []
    
    for i in range(ep.shape[0] - 1):
        if not to_merge[i]:
            new_ends.append(ep[i, 1])
            new_starts.append(ep[i+1, 0])

    new_ends.append(ep[-1, 1])
    return np.array(np.hstack(
            (np.array(new_starts)[..., np.newaxis],
             np.array(new_ends)[..., np.newaxis])))

print(merge(ep1, gap=-0.11))

In [ ]:
y = np.array([[0.0, 1.0],
              [1.6, 2.0],
              [0.9, 1.5]])

In [ ]:
y = np.sort(y, axis=0)
print(type(y))

In [ ]:
times = np.array([0, 2, 4, 10])
dur = np.array([2.5, 0.5, 0.5, 0.5])

In [ ]:
stop_epoch = times + dur
times = np.hstack((times[..., np.newaxis], stop_epoch[..., np.newaxis]))
print(times.shape)
print(dur.shape)

In [ ]:
times

In [ ]:
y = np.array([[  -1. ,   1.5],
              [  3. ,   4.7]])

In [ ]:
y

In [ ]:
intersects = []
for time in times:
    for e in y:
        if (time[1] >= e[0] and time[0] <= e[0]) or (time[1] >= e[1] and time[0] > e[0]) or (time[1] <= e[0] and time[1] < e[0]):
            intersects.append(time)
print(intersects)